In [13]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, ActivityRegularization, Input, Reshape, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import plot_model, to_categorical
from PIL import Image
from astropy.nddata.utils import Cutout2D
from scipy import ndimage
import sys
import time

In [14]:
class BubbleEvent:
    def __init__(self, File):
        #temp pixel arrays and event level meta data
        self.FileName = File
        Bot1PixelArray, Bot2PixelArray = BubbleEvent.GetPixelArray(self.FileName) #gets 2d array of pixel intensities
        self.Date, self.Run, self.EventID = BubbleEvent.GetRunInfo(self.FileName) #parses image name to get event info
        self.BubbleCount = 0
        #actual features to use to classify
        self.UsefulEdgeFeature0, self.UsefulEdgeFeature1, self.UsefulEdgeFeature2 = (GetEdgeFeature(
                                        DownSampleTheArray(2, Bot1PixelArray)) + 
                                        GetEdgeFeature(DownSampleTheArray(2, Bot2PixelArray))) #edge detect. sum
        self.UsefulBlobFeature = np.std(GetBlobs(Bot1PixelArray)) + np.std(GetBlobs(Bot2PixelArray)) #blob convalution deviation
        self.CountBlobPeakFeature = GetPeaks(Bot1PixelArray) + GetPeaks(Bot2PixelArray)
    def GetPixelArray(FileName):
        im = Image.open(FileName)
        PixelArray = np.asarray(im)
        Cutout = Cutout2D(PixelArray, (530,140), 235) #just cut out the parts of the image with bottles
        Bot1PixelArray = Cutout.data
        PixelArray =ndimage.rotate(PixelArray, -45)
        Cutout2 = Cutout2D(PixelArray, (270,310), 235) #other bottle view
        Bot2PixelArray = Cutout2.data
        return Bot1PixelArray, Bot2PixelArray
    def GetRunInfo(File):
        Date = int(File.split("/")[-1].split("_")[0]) #file should be date_run_event
        Run = int(File.split("/")[-1].split("_")[1])
        Event = int("{}{}{}".format(Date, Run,File.split("/")[-1].split("_")[2])) 
        return Date, Run, Event

def OneHotEncode(Y):
    NewY = np.zeros((len(Y), len(set(Y))))
    for Count in set(Y):
        Cut = Y==Count
        NewY[Cut, Count] = 1
    return NewY

def BuildBasicModel(Weights, X_Train, Y_Train):
    loss_fn = 'categorical_crossentropy'
    optim = 'adam'
    batch_size = 1000
    basic_model = Sequential()
    basic_model.add(Input(len(X_Train[0])))
    basic_model.add(Dense(6, activation='relu'))
    basic_model.add(Dropout(rate = 0.2))
    basic_model.add(Dense(7, activation='relu'))
    basic_model.add(Dense(len(set(Y)), activation='softmax'))
    basic_model.compile(optimizer = optim,
                        loss = loss_fn,
                        metrics = ['accuracy'])
    history = basic_model.fit(X_Train, 
                    Y_Train, 
                    batch_size = batch_size,
                    epochs = 150, class_weight=Weights)
    return basic_model

def TestBasicModel(basic_model, X_Test, Y_Test):
    print("Overall Accuracy")
    Score = basic_model.evaluate(X_Test, Y_Test, verbose=0)
    print(Score)
    Preds = basic_model.predict(X_Test)
    ConfusionMatrix = np.zeros((len(set(Y)),len(set(Y))))
    i=0
    for Correct in (set(Y)):
        j=0
        for Guess in set(Y):
            Cut = (np.argmax(Y_Test, axis=1) == Correct)*(np.argmax(Preds, axis=1) == Guess)
            ConfusionMatrix[i,j] = Cut.sum()
            j=j+1
        i=i+1
    print("Confusion Matrix")
    print(ConfusionMatrix)
    print(np.argmax(ConfusionMatrix, axis=1))
    print("Accuracy in each category")
    print(0, ConfusionMatrix[0,0]/np.sum(ConfusionMatrix[0,:]))
    print(1, ConfusionMatrix[1,1]/np.sum(ConfusionMatrix[1,:]))
    print(2, ConfusionMatrix[2,2]/np.sum(ConfusionMatrix[2,:]))
    print(3, ConfusionMatrix[3,3]/np.sum(ConfusionMatrix[3,:]))
    print(4, ConfusionMatrix[4,4]/np.sum(ConfusionMatrix[4,:]))

# Build a basic model using my features

In [16]:
EventData = pkl.load(open("Events.p", "rb"))
X=[]
Y=[]
for i in range(0, len(EventData)):
    if(EventData[i].BubbleCount != -1 or EventData[i].CountBlobPeakFeature == 0): #no visible bubbles or good count
        if(EventData[i].BubbleCount>0):
            Y.append(EventData[i].BubbleCount)
        else:
            Y.append(0)
        X.append([EventData[i].UsefulEdgeFeature0,EventData[i].UsefulEdgeFeature1,EventData[i].UsefulEdgeFeature2, 
                  EventData[i].UsefulBlobFeature, EventData[i].CountBlobPeakFeature])
X = np.asarray(X)
X[:,0] = (np.mean(X[:,0]) - X[:,0])/np.std(X[:,0])
X[:,1] = (np.mean(X[:,1]) - X[:,1])/np.std(X[:,1])
X[:,2] = (np.mean(X[:,2]) - X[:,2])/np.std(X[:,2])
Y = np.asarray(Y)
NewY = OneHotEncode(Y)

In [20]:
X_Train = X[0:14000, :]
Y_Train = NewY[0:14000, :]
X_Test = X[14000:, :]
Y_Test = NewY[14000:, :]

Weights = {0:float(1)/(Y==0).sum(),1:float(1)/(Y==1).sum(), 
           2: float(1)/(Y==2.0).sum(), 3:float(1)/(Y==3.0).sum(), 4:float(1)/(Y==4.0).sum() }

#Weights = {0:1,1:1, 2: 1, 3:1, 4:1 }

basic_model = BuildBasicModel(Weights, X_Train, Y_Train)
TestBasicModel(basic_model, X_Test, Y_Test)

Epoch 1/150
14/14 [==============================] - 0s 4ms/step - loss: 4.7947e-04 - accuracy: 0.0219
Epoch 2/150
14/14 [==============================] - 0s 3ms/step - loss: 4.2064e-04 - accuracy: 0.0252
Epoch 3/150
14/14 [==============================] - 0s 3ms/step - loss: 4.7229e-04 - accuracy: 0.0281
Epoch 4/150
14/14 [==============================] - 0s 3ms/step - loss: 4.0353e-04 - accuracy: 0.0302
Epoch 5/150
14/14 [==============================] - 0s 3ms/step - loss: 3.8567e-04 - accuracy: 0.0361
Epoch 6/150
14/14 [==============================] - 0s 3ms/step - loss: 4.4190e-04 - accuracy: 0.0376
Epoch 7/150
14/14 [==============================] - 0s 3ms/step - loss: 3.8696e-04 - accuracy: 0.0404
Epoch 8/150
14/14 [==============================] - 0s 3ms/step - loss: 4.1919e-04 - accuracy: 0.0476
Epoch 9/150
14/14 [==============================] - 0s 3ms/step - loss: 4.2865e-04 - accuracy: 0.0519
Epoch 10/150
14/14 [==============================] - 0s 3ms/step - loss:

14/14 [==============================] - 0s 4ms/step - loss: 2.4174e-04 - accuracy: 0.7286
Epoch 80/150
14/14 [==============================] - 0s 4ms/step - loss: 2.4699e-04 - accuracy: 0.7335
Epoch 81/150
14/14 [==============================] - 0s 3ms/step - loss: 2.9749e-04 - accuracy: 0.7334
Epoch 82/150
14/14 [==============================] - 0s 3ms/step - loss: 2.4970e-04 - accuracy: 0.7336
Epoch 83/150
14/14 [==============================] - 0s 3ms/step - loss: 2.5297e-04 - accuracy: 0.7380
Epoch 84/150
14/14 [==============================] - 0s 3ms/step - loss: 2.5717e-04 - accuracy: 0.7298
Epoch 85/150
14/14 [==============================] - 0s 3ms/step - loss: 2.5339e-04 - accuracy: 0.7304
Epoch 86/150
14/14 [==============================] - 0s 3ms/step - loss: 2.2939e-04 - accuracy: 0.7306
Epoch 87/150
14/14 [==============================] - 0s 4ms/step - loss: 2.6560e-04 - accuracy: 0.7299
Epoch 88/150
14/14 [==============================] - 0s 4ms/step - loss: 2.3

/home/jacob/anaconda2/envs/mypython3/lib/python3.7/site-packages/ipykernel_launcher.py:77: RuntimeWarning: invalid value encountered in double_scalars


# Try a convalutional thing instead

In [18]:
def GetPixelArray(FileName):
        im = Image.open(FileName)
        PixelArray = np.asarray(im)
        Cutout = Cutout2D(PixelArray, (530,140), 235) #just cut out the parts of the image with bottles
        Bot1PixelArray = Cutout.data
        PixelArray =ndimage.rotate(PixelArray, -45)
        Cutout2 = Cutout2D(PixelArray, (270,310), 235) #other bottle view
        Bot2PixelArray = Cutout2.data
        return Bot1PixelArray, Bot2PixelArray

def OtherDownSampleArray(BoxSize, Array):
    TempArray = Array[:np.shape(Array)[0]-np.shape(Array)[0]%BoxSize, 
                      :np.shape(Array)[1]-np.shape(Array)[1]%BoxSize] #cut away last rows/columns if needed
    Dim = np.shape(TempArray)[0]
    NewArray = TempArray.reshape(Dim//BoxSize, BoxSize, Dim//BoxSize, BoxSize).mean(3).mean(1)
    return NewArray

Array1, Array2 = GetPixelArray(EventData[0].FileName)
OneEventByteSize = sys.getsizeof(Array1) + sys.getsizeof(Array2)
FreeRam = 12e9
print(FreeRam/OneEventByteSize)
#should be able to read in all events easy enough!
Start=time.time()
UsefulArrays = np.zeros((len(EventData), 2, int(np.shape(Array1)[0]/2), int(np.shape(Array1)[1]/2)))
#UsefulArrays = np.zeros((3, 2, int(np.shape(Array1)[0]/2), int(np.shape(Array1)[1]/2)))
#for i in range(0, len(EventData)):
for i in range(0, 3):
    Array1, Array2 = GetPixelArray(EventData[i].FileName)
    TempArray1 = OtherDownSampleArray(2, Array1)
    TempArray2 = OtherDownSampleArray(2, Array2)
    UsefulArrays[i,0] = TempArray1
    UsefulArrays[i,1] = TempArray2

53571428.571428575


In [12]:
#UsefulArrays = UsefulArrays/255.0 #scale the pixel values down
loss_fn = 'categorical_crossentropy'
optim = 'adam'
batch_size = 500
print(np.shape(UsefulArrays))
conv_model = Sequential()
conv_model.add(Input((2, 117,117)))
conv_model.add(Reshape((117, 117*2, 1)))
conv_model.add(Conv2D(32, kernel_size=(3, 3)))
conv_model.add(Conv2D(64, (3, 3)))
conv_model.add(Activation('relu'))
conv_model.add(MaxPooling2D(pool_size=(4, 4)))
conv_model.add(Dropout(0.2))
conv_model.add(Flatten())
conv_model.add(Dense(128))
conv_model.add(Activation('relu'))
conv_model.add(Dropout(0.5))
conv_model.add(Dense(len(set(Y))))
conv_model.add(Activation('softmax'))
conv_model.compile(optimizer = optim,
              loss = loss_fn,
              metrics = ['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', 
                                            patience=10, 
                                            restore_best_weights = True)

conv_model.fit(UsefulArrays[0:14000], 
          Y_Train, 
          batch_size = batch_size,
          epochs = 50,
          validation_split = 0.1,
          callbacks = [callback])
TestBasicModel(conv_model, UsefulArrays[14000:], Y_Test)

(3, 2, 117, 117)
(2, 2, 117, 117) (2, 5)
Epoch 1/50
1/1 [==============================] - 0s 173ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 38ms/step - loss: 18.3601 - accuracy: 0.0000e+00 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 33ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 37ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/50
1/1 [==============================] - 0s 34ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 36ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 34ms/step - loss: 0.00

ValueError: Data cardinality is ambiguous:
  x sizes: 0
  y sizes: 7411
Please provide data which shares the same first dimension.